<a href="https://colab.research.google.com/github/jcbdoliveira/DIO-ETL-Santander/blob/main/Desafio_DIO_ETL_Santander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio DIO Santander Bootcamp (ETL com Python)




**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de somar os pontos conquistados pelos clientes. Seu objetivo é usar o Python para ler três arquivos CSV, identificar as transações de cada cliente e somar os pontos obtidos. Depois você deve usar poder da IA Generativa para criar mensagens personalizadas que serão entregues a cada cliente. Se o cliente possuir mais de 100 pontos deve ser uma mensagem de parabéns, se estiver entre 80 e 99, deve ser uma mensagem de "quase lá", se estiver entre 20 e 79, a mensagem deve ser de incetivo, mas se estiver abaixo de 20 a mensagem deve demonstrar as qualidades do banco e convidar o cliente para utilizar mais serviços.
Cada mensagem vai ser entregue por e-mail para o cliente.

**Condições do Problema:**

1. Você recebeu os arquivos:
users.csv
transactions.csv
transaction_types.csv
  ```
2. O trabalho começa lendo os dados e relacionando as informações.
3. Na sequencia apuramos os pontos que cada cliente conquistou e armazenamos dentro de uma nova coluna.
4. Consumimos a API do chatGPT e armazemanos as mensagems dentro do dataframe.
5. Por fim lemos o nosso dataframe final e enviamos os e-mails personalizados.


In [17]:
usuarios = 'users.csv'
extrato = 'transactions.csv'
pontos = 'transaction_types.csv'

## **E**xtract

Ler e extrair os dados dos arquivos e carregar dentro dos dataframes.

In [18]:
import pandas as pd
import numpy as np

dfUsusarios = pd.read_csv(usuarios, sep=';', header=0, index_col='user_id')
dfExtrato = pd.read_csv(extrato, sep=';', header=0)
dfPontos = pd.read_csv(pontos, sep=';', header=0)
dfUsusarios['pontos'] = 0
dfUsusarios['mensagem_email'] = ''

print(dfUsusarios.head())
print(dfExtrato.head())
print(dfPontos.head())

dfUsusarios = dfUsusarios.reset_index()

def soma_pontos(user,tipo):
   valor_lido = 0
   valor_lido = dfPontos.loc[dfPontos['transaction_type']==tipo, 'points']
   dfUsusarios.loc[dfUsusarios['user_id'] == user, 'pontos'] += np.int64(valor_lido)

dfExtrato = dfExtrato.reset_index()
for index, row in dfExtrato.iterrows():
    soma_pontos(row['user_id'], row['transaction_type'])

for index, row in dfUsusarios.iterrows():
    print(row['user_id'], row['user_name'], row['pontos'], row['mensagem_email'])

                 user_name                   user_email  pontos mensagem_email
user_id                                                                       
1              Julio Cesar  julio@adminformatica.com.br       0               
3        Venilton Falvo Jr                ***@gmail.com       0               
2              Julio Gmail        jcboliveira@gmail.com       0               
   user_id  transaction_id transaction_type    value
0        1            1001               A1   100.00
1        1            1002               A1    50.00
2        1            1003               A5  1000.00
3        1            1004               A3    50.92
4        1            1005               A1   100.00
  transaction_type  points
0               A1       1
1               A2       2
2               A3       3
3               A4       4
4               A5       5
1 Julio Cesar 20 
3 Venilton Falvo Jr 32 
2 Julio Gmail 145 


## **T**ransform

Utilize a API do OpenAI GPT-3.5 para gerar uma mensagem de marketing personalizada para cada usuário e prepare para envio por e-mail.

In [19]:
!pip install openai

In [20]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = 'TODO'

In [21]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user, pontos):
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
          "role": "system",
          "content": "Você é especialista em marketing bancário."
      },
      {
          "role": "user",
          "content": f"Crie um e-mail para o cliente {user} que possui {pontos} pontos em nosso programa de fidelidade. Se o cliente possuir mais de 100 pontos deve ser criar uma mensagem de parabéns por ter alcançado. Se os pontos estivem entre 80 e 99, deve ser uma mensagem de estímulo. Se os pontos estivem entre 20 e 79, a mensagem deve ser de incetivo e engajamento ao programa. Se os pontos forem menor ou igual 20 a mensagem deve demonstrar as qualidades do Banco Santander e convidar o cliente para utilizar mais serviços e produtos. Deve ter no máximo de 250 caracteres e ser assinado pela equipe de marketing do Banco Santander."
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for index, row in dfUsusarios.iterrows():
  news = generate_ai_news(row['user_name'], row['pontos'])
  dfUsusarios.loc[dfUsusarios['user_id'] == row['user_id'], 'mensagem_email'] = news
  print(news)
  print('++++++++++++++++++++++++++')


Assunto: Parabéns, Julio Cesar!

Caro Julio Cesar,

Parabenizamos você por ter alcançado 20 ponto(s) em nosso programa de fidelidade! Estamos ansiosos para proporcionar ainda mais vantagens e recompensas exclusivas.

Continue assim e em breve alcançará novos patamares. Estamos aqui para apoiá-lo em sua jornada.

Conte sempre com o Banco Santander!

Atenciosamente,
Equipe de Marketing do Banco Santander
++++++++++++++++++++++++++
Assunto: Parabéns pelos seus 32 pontos em nosso programa de fidelidade!

Prezado Sr. Venilton Falvo Jr,

Parabenizamos você por acumular 32 pontos em nosso programa de fidelidade. Continue utilizando nossos serviços e produtos e aproveite os benefícios exclusivos oferecidos.

Equipe de Marketing do Banco Santander
++++++++++++++++++++++++++
Prezado Julio Gmail, 

Parabéns por alcançar 145 pontos em nosso programa de fidelidade! Continue assim e aproveite os benefícios exclusivos que preparamos para você. 

Atenciosamente, 
Equipe de Marketing Banco Santander
++

## **L**oad

Percorre a lista de usuários e envia a mensagem de marketing gerada pelo ChatGPT.

In [22]:
remetente = '***@gmail.com'

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Configuração
host = 'smtp.gmail.com'
port = 587
user = '***@gmail.com'
password = 'TODO'

# Criando objeto
print('Criando objeto servidor...')
server = smtplib.SMTP(host, port)

# Login com servidor
print('Login...')
server.ehlo()
server.starttls()
server.login(user, password)

for index, row in dfUsusarios.iterrows():
  # Criando mensagem
  message = row['mensagem_email']
  print('Criando mensagem...')
  email_msg = MIMEMultipart()
  email_msg['From'] = remetente
  email_msg['To'] = row['user_email']
  email_msg['Subject'] = 'Santander Bootcamp 2023 - Ciência de dados com Python'
  print('Adicionando texto...')
  email_msg.attach(MIMEText(message, 'plain'))

  # Enviando mensagem
  print('Enviando mensagem...')
  server.sendmail(email_msg['From'], email_msg['To'], email_msg.as_string())
  print('Mensagem enviada!')

server.quit()

Criando objeto servidor...
Login...
Criando mensagem...
Adicionando texto...
Enviando mensagem...
Mensagem enviada!
Criando mensagem...
Adicionando texto...
Enviando mensagem...
Mensagem enviada!
Criando mensagem...
Adicionando texto...
Enviando mensagem...
Mensagem enviada!


(221,
 b'2.0.0 closing connection gw15-20020a17090b0a4f00b0026f90d7947csm7691890pjb.34 - gsmtp')